In [2]:
import pandas as pd
import numpy as np
import os
import re

In [8]:
curtest = pd.DataFrame()
for year in range(10, 23):  # range is exclusive at the end
    filename = f"/Users/danielmatten/Desktop/m/curtest_pub20{year}.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values

        
        # Append to the main DataFrame
        curtest = pd.concat([curtest, df], ignore_index=True)
        print("{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")

def clean_bytes(val):
    if isinstance(val, bytes):
        return str(val)[2:-1]  # str(b'xyz') => "b'xyz'" → "xyz"
    return val

curtest.drop(columns=['accomm_list', 'admindt','test_lea','test_schl','lea','schlcode','testdt','collection_code','exemption_code','ach_level','grade'], inplace=True) #really wonder if accomm_list is interesting

for col in curtest.columns:
    curtest[col] = curtest[col].apply(clean_bytes)
    print(col)

curtest.to_csv("../data/curtest_master.csv",index=False)

{year} done
{year} done
{year} done
{year} done
{year} done
{year} done
{year} done
{year} done
{year} done
{year} done
File /Users/danielmatten/Desktop/m/curtest_pub2020.sas7bdat not found. Skipping.
{year} done
{year} done
year
test_id
exempt
score
cscore
mastid
pc


In [4]:
curtest = pd.read_csv("../data/curtest_master.csv")
exit_ids = pd.read_csv("../data/exit_list.csv")

/var/folders/dk/2mw0y5w91dggrsnrpxb04jv80000gn/T/ipykernel_43893/3725179964.py:1: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  curtest = pd.read_csv("../data/curtest_master.csv")


In [19]:
curtest_copy = curtest.copy()

In [21]:
curtest_copy['score'] = pd.to_numeric(curtest_copy['score'], errors='coerce')

curtest_copy.dropna(subset=['score'])
# Rank scores within teach (test_id, year) group
curtest_copy['pc'] = curtest_copy.groupby(['test_id', 'year'])['score'].rank(pct=True) * 100


In [22]:
curtest_copy = curtest_copy[curtest_copy['mastid'].isin(exit_ids['mastid'])]

In [23]:
curtest_copy.drop(columns=['year','score','cscore','exempt'],inplace=True)
curtest_copy.dropna(inplace=True)

In [24]:
# Step 1: Average pc for each mastid-test_id combo
avg_pc = curtest_copy.groupby(['mastid', 'test_id'], as_index=False)['pc'].mean()

# Step 2: Drop duplicates from the original
curtest_copy = curtest_copy.drop_duplicates(subset=['mastid', 'test_id'])

# Step 3: Update the pc column with the averaged values
curtest_copy = curtest_copy.drop(columns='pc').merge(avg_pc, on=['mastid', 'test_id'], how='left')


In [29]:
curtest_copy['test_id'].unique()

array(['MA07', 'RD07', 'MA05', 'RD05', 'SC05', 'MA04', 'RD04', 'MA06',
       'RD06', 'MA03', 'RD03', 'MA08', 'RD08', 'SC08', 'CECO', 'ENGL',
       'BIOL', 'ENG2', 'ACCO', 'MTH1', 'MTH3'], dtype=object)

In [42]:
curtest_copy = curtest_copy[
    ~((curtest_copy['test_id'].str.startswith(('X', 'A','RD3','C','E','MTH3','MA03','RD03'))) & (curtest_copy['test_id'] != 'ACCO'))
]


In [43]:
pivot_df = curtest_copy.pivot(index='mastid', columns='test_id', values='pc')
pivot_df.columns = [f'pc_{col}' for col in pivot_df.columns]
pivot_df = pivot_df.reset_index()


In [44]:
# Count number of NaNs per row
na_counts = pivot_df.isna().sum(axis=1)

# Count how many rows have exactly X NaNs
missing_summary = na_counts.value_counts().sort_index()

print("Number of rows by count of missing values:")
print(missing_summary)


Number of rows by count of missing values:
0     232393
1     127665
2      50768
3      15203
4       7569
5      12348
6       8796
7      11242
8       7468
9       9698
10      6873
11      2656
12     10870
13     15773
14     16394
Name: count, dtype: int64


In [45]:
pivot_df2 = pivot_df[pivot_df.isna().sum(axis=1) <= 2]


In [46]:
pivot_df2.to_csv("../data/curtest_mastid.csv",index=False)